In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer
import mmcv

In [ ]:
# Pad naar de config en checkpoint
config_file = r'C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\mmaction2\configs\recognition\tsm\tsm_imagenet-pretrained-r50_8xb16-1x1x8-50e_sthv2-rgb.py'
checkpoint_file = r'C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\mmaction2\checkpoints\tsm_imagenet-pretrained-r50_8xb16-1x1x8-50e_sthv2-rgb_20230317-be0fc26e.pth'

In [ ]:
# Model initialiseren
model = init_recognizer(config_file, checkpoint_file, device='cuda:0')  # Gebruik 'cpu' als je geen GPU hebt

In [ ]:
# Voer inferentie uit op een sample video
video_path = r'C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Video zonder titel.mp4'  # Zorg ervoor dat je een video bestand hebt
results = inference_recognizer(model, video_path)

In [ ]:
# Laad de labelmap
labels = open('tools/data/sthv2/label_map.txt').read().splitlines()

In [ ]:
import torch

# Converteer de tensor naar een Python lijst
pred_labels = results.pred_label.cpu().numpy()  # Voorspelde labels
pred_scores = results.pred_score.cpu().numpy()  # Bijbehorende scores

# Sorteer en pak de top-5 voorspellingen
top5_indices = torch.topk(results.pred_score, 5).indices.cpu().numpy()

# Laad de labelmap
with open('tools/data/sthv2/label_map.txt') as f:
    labels = f.read().splitlines()

# Converteer indices naar labels
top5_labels = [labels[i] for i in top5_indices]

print(f"Top-5 voorspellingen: {top5_labels}")


In [ ]:
import time

start_time = time.time()

In [ ]:
import os
import cv2
import torch
import mmcv
import tempfile
import numpy as np

from mmaction.apis import inference_recognizer, init_recognizer
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
# config_file = r'C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\mmaction2\configs\recognition\tsm\tsm_imagenet-pretrained-r50_8xb16-1x1x8-50e_sthv2-rgb.py'
# checkpoint_file = r'C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\mmaction2\checkpoints\tsm_imagenet-pretrained-r50_8xb16-1x1x8-50e_sthv2-rgb_20230317-be0fc26e.pth'
config_file = r'C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\mmaction2\configs\recognition\tsm\tsm_imagenet-pretrained-r101_8xb16-1x1x8-50e_sthv2-rgb.py'
checkpoint_file = r'C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\mmaction2\checkpoints\tsm_imagenet-pretrained-r101_8xb16-1x1x8-50e_sthv2-rgb_20230320-efcc0d1b.pth'
video_path = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Video's\machine cover.mp4"
label_map_path = 'tools/data/sthv2/label_map.txt'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# --- Initieer model ---
model = init_recognizer(config_file, checkpoint_file, device=device)
target_layers = [model.backbone.layer4[-1]]

In [ ]:
# --- Labels inladen ---
with open(label_map_path, 'r') as f:
    labels = f.read().splitlines()

In [ ]:
# --- Functie om frames naar video te schrijven ---
def frames_to_video_cv2(frames, out_path, fps=25):
    if not frames:
        return
    height, width = frames[0].shape[:2]
    writer = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    for frame in frames:
        writer.write(frame)
    writer.release()

In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from mmcv.transforms import Compose
from mmengine import Config

# def get_transforms_from_cfg(cfg_path):
#     cfg = Config.fromfile(cfg_path)
#     return Compose(cfg.test_pipeline)

In [ ]:
import torch.nn as nn

class CAMWrapper(nn.Module):
    def __init__(self, backbone, cls_head, num_segs=8):
        super().__init__()
        self.backbone = backbone
        self.cls_head = cls_head
        self.num_segs = num_segs

    def forward(self, x):
        x = x.view(-1, 3, 224, 224)  # [N*T, C, H, W]
        feat = self.backbone(x)     # [N*T, 2048]
        out = self.cls_head(feat, self.num_segs)  # [N, num_classes]
        return out



In [ ]:
import torchvision.transforms as T
# Hoofdfunctie om Grad-CAM + bounding box te genereren
def generate_gradcam_heatmap(model, video_path, target_label, target_layers):
    reader = mmcv.VideoReader(video_path)
    frames = list(reader[:8])
    if len(frames) < 8:
        return None

    # Originele resolutie opslaan
    mid_frame = frames[4]
    orig_h, orig_w = mid_frame.shape[:2]

    # Preprocessing: resize + normaliseren (224x224)
    transform = T.Compose([
        T.ToPILImage(),
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
    ])
    processed_frames = [transform(f) for f in frames]  # [C,H,W] * 8
    clip = torch.stack(processed_frames, dim=0)         # [T,C,H,W]
    input_tensor = clip.to(device).float()              # [T,C,H,W]
    input_tensor = input_tensor.reshape(-1, 3, 224, 224)  # [8,3,224,224]

    # Model in wrapper voor Grad-CAM
    cam_model = CAMWrapper(model.backbone, model.cls_head, num_segs=8).to(device)
    cam = GradCAM(model=cam_model, target_layers=target_layers)

    # Grad-CAM genereren
    targets = [ClassifierOutputTarget(target_label)]
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]  # [224,224]

    # CAM visualiseren op resized RGB frame
    frame_resized = cv2.resize(mid_frame, (224, 224))
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    cam_image = show_cam_on_image(frame_rgb, grayscale_cam, use_rgb=True)

    # Bounding box uit CAM
    threshold = 0.5
    cam_mask = (grayscale_cam > threshold).astype(np.uint8) * 255
    contours, _ = cv2.findContours(cam_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bounding_box_coords = None  # fallback

    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)

        # Schaal terug naar originele framegrootte
        scale_x = orig_w / 224
        scale_y = orig_h / 224
        x_orig = int(x * scale_x)
        y_orig = int(y * scale_y)
        w_orig = int(w * scale_x)
        h_orig = int(h * scale_y)

        # Bounding box tekenen op originele frame (optioneel debug)
        mid_frame_boxed = mid_frame.copy()
        cv2.rectangle(mid_frame_boxed, (x_orig, y_orig), (x_orig + w_orig, y_orig + h_orig), (0, 255, 0), 2)

        # Eventueel: bounding box opslaan
        bounding_box_coords = {
            "x": x_orig,
            "y": y_orig,
            "w": w_orig,
            "h": h_orig,
            "frame_size": (orig_w, orig_h)
        }

        print("Bounding box op origineel frame:", bounding_box_coords)

        # Box ook tekenen op CAM (resized)
        cam_image = cv2.rectangle(cam_image, (x, y), (x + w, y + h), color=(0, 255, 0), thickness=2)

    return cam_image, bounding_box_coords


In [ ]:
# --- Verwerk video in snippets ---
video_reader = mmcv.VideoReader(video_path)
fps = video_reader.fps
total_frames = len(video_reader)

In [ ]:
snippet_length_sec = 0.5
overlap_sec = 0.25
snippet_length = int(snippet_length_sec * fps)
overlap = int(overlap_sec * fps)
step = snippet_length - overlap

In [ ]:
segment_results = []
temp_dir = tempfile.gettempdir()

for start_frame in range(0, total_frames, step):
    end_frame = start_frame + snippet_length
    if end_frame > total_frames:
        break

    snippet_frames = video_reader[start_frame:end_frame]
    if not snippet_frames:
        continue

    snippet_name = f"temp_snippet_{start_frame}_{end_frame}.mp4"
    snippet_path = os.path.join(temp_dir, snippet_name)
    frames_to_video_cv2(snippet_frames, snippet_path, fps=fps)

    result = inference_recognizer(model, snippet_path)
    pred_score = result.pred_score
    pred_label = result.pred_label
    top5 = torch.topk(pred_score, 5)
    top5_indices = top5.indices.cpu().numpy()
    top5_scores = top5.values.cpu().numpy()
    top5_labels = [labels[idx] for idx in top5_indices]

    best_label_idx = pred_label.cpu().item()
    best_label = labels[best_label_idx]
    best_score = pred_score[best_label_idx].cpu().item()
    start_sec = start_frame / fps
    end_sec = end_frame / fps

    # --- Grad-CAM genereren ---
    cam_img, bbox = generate_gradcam_heatmap(model, snippet_path, best_label_idx, target_layers)
    if cam_img is not None:
        output_img_path = os.path.join(temp_dir, f"gradcam_{start_frame}_{end_frame}.jpg")
        cv2.imwrite(output_img_path, cam_img)
        print(f"Heatmap opgeslagen: {output_img_path}")

    # --- Segment opslaan ---
    segment_results.append({
        'start_time': start_sec,
        'end_time': end_sec,
        'best_label': best_label,
        'best_score': best_score,
        'top5_labels': top5_labels,
        'top5_scores': top5_scores,
        'bounding_box': bbox
    })

In [ ]:
final_segments = []
current_label = None
current_start_sec = None
current_end_sec = None
current_scores = []
current_bboxes = []

score_threshold = 0.6

for seg in segment_results:
    label = seg['best_label']
    score = seg['best_score']
    bbox = seg.get('bounding_box')  # kan None zijn

    if score < score_threshold:
        continue

    if current_label is None:
        current_label = label
        current_start_sec = seg['start_time']
        current_end_sec = seg['end_time']
        current_scores.append(score)
        if bbox:
            current_bboxes.append(bbox)
        continue

    if label == current_label and seg['start_time'] <= current_end_sec:
        current_end_sec = max(current_end_sec, seg['end_time'])
        current_scores.append(score)
        if bbox:
            current_bboxes.append(bbox)
    else:
        avg_score = sum(current_scores) / len(current_scores)

        # ➕ Gemiddelde bounding box berekenen (als er bboxen zijn)
        if current_bboxes:
            avg_bbox = {
                'x': int(np.mean([b['x'] for b in current_bboxes])),
                'y': int(np.mean([b['y'] for b in current_bboxes])),
                'w': int(np.mean([b['w'] for b in current_bboxes])),
                'h': int(np.mean([b['h'] for b in current_bboxes])),
                'frame_size': current_bboxes[0]['frame_size']
            }
        else:
            avg_bbox = None

        final_segments.append({
            'label': current_label,
            'start': current_start_sec,
            'end': current_end_sec,
            'avg_score': avg_score,
            'avg_bounding_box': avg_bbox
        })

        # Nieuw segment starten
        current_label = label
        current_start_sec = seg['start_time']
        current_end_sec = seg['end_time']
        current_scores = [score]
        current_bboxes = [bbox] if bbox else []

# Vergeet laatste segment niet
if current_label is not None:
    avg_score = sum(current_scores) / len(current_scores)

    if current_bboxes:
        avg_bbox = {
            'x': int(np.mean([b['x'] for b in current_bboxes])),
            'y': int(np.mean([b['y'] for b in current_bboxes])),
            'w': int(np.mean([b['w'] for b in current_bboxes])),
            'h': int(np.mean([b['h'] for b in current_bboxes])),
            'frame_size': current_bboxes[0]['frame_size']
        }
    else:
        avg_bbox = None

    final_segments.append({
        'label': current_label,
        'start': current_start_sec,
        'end': current_end_sec,
        'avg_score': avg_score,
        'avg_bounding_box': avg_bbox
    })


In [ ]:
# Resultaten printen
for fs in final_segments:
    print(f"Actie '{fs['label']}' van {fs['start']:.2f}s tot {fs['end']:.2f}s (Gemiddelde score: {fs['avg_score']:.2f})")

In [ ]:
import json
output_json_path = os.path.join(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\mmaction2", "final_segments_with_boxes_machine_cover_2.json")

with open(output_json_path, 'w') as f:
    json.dump(final_segments, f, indent=4)

print(f"JSON-bestand opgeslagen: {output_json_path}")


In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Totaal uitgevoerde tijd: {elapsed_time:.2f} seconden")

In [ ]:
from mmcv import VideoReader
import cv2

def annotate_video_with_segments(video_path, segments, output_path, font_scale=0.6):
    vr = VideoReader(video_path)
    fps = vr.fps
    width, height = vr.resolution
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame_idx, frame in enumerate(vr):
        current_time = frame_idx / fps
        annotated_frame = frame.copy()

        for seg in segments:
            if seg['start'] <= current_time <= seg['end']:
                label = seg['label']
                bbox = seg.get('avg_bounding_box')
                if bbox:
                    x, y, w, h = bbox['x'], bbox['y'], bbox['w'], bbox['h']
                    cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(
                        annotated_frame,
                        f"{label}",
                        (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        font_scale,
                        (0, 255, 0),
                        2
                    )
        writer.write(annotated_frame)

    writer.release()
    print(f"Annotated video opgeslagen: {output_path}")


In [ ]:
annotated_video_path = os.path.join(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\mmaction2", "annotated_output_stabilizer_pressure_control.mp4")
annotate_video_with_segments(video_path, final_segments, annotated_video_path)
